Pontifícia Universidade Católica de São Paulo 

`Ciência de Dados e Inteligência Artificial`

🎓 Laboratório 10 - Support Vector Machines

13 de outubro de 2022


---
> 👨‍🏫*Professor Rooney Coelho (rracoelho@pucsp.br)*
---




Neste aula você usará SVM (Support Vector Machines) para construir e treinar um modelo usando registros de células humanas e classificar as células se as amostras são benignas ou malignas.

O SVM funciona mapeando dados para um espaço de recursos de alta dimensão para que os pontos de dados possam ser categorizados, mesmo quando os dados não são linearmente separáveis. Um separador entre as categorias é encontrado, então os dados são transformados de tal forma que o separador pode ser desenhado como um hiperplano. Em seguida, as características dos novos dados podem ser usadas para prever o grupo ao qual um novo registro deve pertencer.

In [23]:
# Dependências
import pandas as pd
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

### Importação e limpeza dos dados

1) Importe os dados da seguinte url: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/cell_samples.csv

In [24]:
url = 'https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/cell_samples.csv'
df = pd.read_csv(url)
df.head()

,ID,Clump,UnifSize,UnifShape,MargAdh,SingEpiSize,BareNuc,BlandChrom,NormNucl,Mit,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


O campo ID contém os identificadores do paciente. As características das amostras de células de cada paciente estão contidas nos campos Clump à Mit. Os valores são classificados de 1 a 10, sendo 1 o mais próximo de benigno.

O campo Class contém o diagnóstico, conforme confirmado por procedimentos médicos separados, se as amostras são benignas (valor = 2) ou malignas (valor = 4).

Vejamos a distribuição das classes com base Clump thickness e Uniformity of cell size:

2) Use o método `info()` para chechar quais colunas possuem valroes não numéricos.

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           699 non-null    int64 
 1   Clump        699 non-null    int64 
 2   UnifSize     699 non-null    int64 
 3   UnifShape    699 non-null    int64 
 4   MargAdh      699 non-null    int64 
 5   SingEpiSize  699 non-null    int64 
 6   BareNuc      699 non-null    object
 7   BlandChrom   699 non-null    int64 
 8   NormNucl     699 non-null    int64 
 9   Mit          699 non-null    int64 
 10  Class        699 non-null    int64 
dtypes: int64(10), object(1)
memory usage: 60.2+ KB


Note que a coluna `BareNuc`possui valores não numéricos. Vamos usar agora o método `unique()`nesta coluna para ver quais são os valores únicos dessa coluna.

In [26]:
df['BareNuc'].unique()

array(['1', '10', '2', '4', '3', '9', '7', '?', '5', '8', '6'],
      dtype=object)

3) Veja que os valores faltantes na coluna `BareNuc`são iguais a `'?'`. Use o método `replace()` em `df` para substituir `'?'` por `np.NaN`. Sobreescreva `df`para não perder a alteração realizada.

In [27]:
import numpy as np

In [28]:
df['BareNuc'] = df['BareNuc'].replace('?', np.nan)

4. Remova agora os dados faltantes usando `dropna()`

In [29]:
df.dropna(inplace=True)

Converta a coluna para `int` usando o método `astype()`.

In [31]:
df['BareNuc'] = df['BareNuc'].astype(int)

### Identificação das features e target

4) Crie um DataFrame `X` com todas as features. Lembrando que o target é o atributo Class e que ID não é uma feature.

In [33]:
X = df[['Clump', 'UnifSize', 'UnifShape', 'MargAdh', 'SingEpiSize', 'BareNuc', 'BlandChrom', 'NormNucl', 'Mit']]

5) Segmente o Dataframe para a representação do Target (Class)

In [34]:
y = df['Class']

## Padronização dos dados

A padronização de recursos é crucial para alguns algoritmos de aprendizado de máquina que consideram distâncias entre observações porque a distância entre duas observações difere para casos não padronizados e padronizados.

Maximizamos no SVM a distância até os pontos de dados mais próximos de diferentes classes. Portanto, a distância calculado entre os pontos de dados afeta o limite de decisão que o SVM escolhe. Em outras palavras, treinar um SVM sobre os dados padronizados e não padronizados leva à geração de modelos diferentes.

6. Use o `StandardScaler()` para padronizar os dados.

In [35]:
from sklearn.preprocessing import StandardScaler

In [36]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

### Separação dos dados para teste e validação.

7) Divida os dados em treino e teste com a função `train_test_split()`. Separe 20% dos dados para validação.

In [38]:
from sklearn.model_selection import train_test_split

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

Verifique se as dimensões dos dados de teste e validação estão coerentes usando o método `shape()`.

In [47]:
print('X_train:', X_train.shape, '    ', 'X_test:', X_test.shape)
print('y_train:', y_train.shape, '       ' 'y_test:', y_test.shape)

X_train: (546, 9)      X_test: (137, 9)
y_train: (546,)        y_test: (137,)


### Treinamento e avaliação da SVM criada

O algoritmo SVM oferece uma escolha de funções de kernel para realizar seu processamento. Basicamente, o mapeamento de dados em um espaço dimensional superior é chamado de kernelling. A função matemática usada para a transformação é conhecida como função kernel e pode ser de diferentes tipos, como:

     1. Linear
     2. Polinômio
     3. Função de base radial (RBF)
     4. Sigmóide
Cada uma dessas funções tem suas características, seus prós e contras e sua equação, mas como não há uma maneira fácil de saber qual função tem o melhor desempenho com qualquer dataset, geralmente escolhemos funções diferentes e comparamos os resultados. 

8) Utilize a Função de base radial (RBF) como kernel. O classificador SVM é representado pelo método `SVC`

In [50]:
from sklearn.svm import SVC

In [51]:
rbf_model = SVC(kernel='rbf', random_state=42)

In [52]:
rbf_model.fit(X_train, y_train)

SVC(random_state=42)

9) Use o método `score()` para avaliar a performance de seu classificador.

In [54]:
y_pred = rbf_model.predict(X_test)

In [60]:
rbf_model.score(X)

ValueError: X has 1 features, but SVC is expecting 9 features as input.

In [58]:
y_test.values

array([2, 2, 2, 2, 4, 4, 2, 2, 2, 4, 4, 4, 4, 4, 4, 2, 2, 2, 2, 2, 4, 2,
       2, 2, 4, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 2, 2, 2, 4, 4, 4, 2,
       2, 4, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 4, 2, 2, 4, 2, 2, 2, 2,
       4, 4, 2, 4, 2, 2, 2, 4, 4, 4, 2, 4, 4, 4, 2, 2, 4, 2, 2, 2, 4, 2,
       2, 2, 4, 4, 4, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 4,
       4, 2, 2, 2, 2, 4, 2, 2, 2, 4, 2, 4, 4, 2, 2, 2, 2, 2, 2, 4, 2, 2,
       2, 4, 4, 2, 4], dtype=int64)